In [1]:
import numpy as np
from qiskit import Aer
from qiskit.transpiler import PassManager

from qiskit_aqua import Operator, QuantumInstance
from qiskit_aqua.algorithms.adaptive import VQE
from qiskit_aqua.algorithms.classical import ExactEigensolver
from qiskit_aqua.components.optimizers import L_BFGS_B
from qiskit_aqua.components.variational_forms import RYRZ

from qiskit_chemistry import FermionicOperator
from qiskit_chemistry.drivers import PySCFDriver, UnitsType

In [2]:
# using driver to get fermionic Hamiltonian
# PySCF example
driver = PySCFDriver(atom='H .0 .0 .0; H .0 .0 0.735', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

In [3]:
# get fermionic operator and mapping to qubit operator
ferOp = FermionicOperator(h1=molecule.one_body_integrals, h2=molecule.two_body_integrals)
qubitOp = ferOp.mapping(map_type='JORDAN_WIGNER', threshold=0.00000001)
qubitOp.to_matrix()
qubitOp.chop(10**-10)

In [4]:
# If you do not install any driver and would like to start with a random Hamiltonian
# SIZE=4
# matrix = np.random.random((SIZE,SIZE))
# qubitOp = Operator(matrix=matrix)

In [5]:
# Using exact eigensolver to get the smallest eigenvalue
exact_eigensolver = ExactEigensolver(qubitOp, k=1)
ret = exact_eigensolver.run()
print('The exact ground state energy is: {}'.format(ret['eigvals'][0].real))

The exact ground state energy is: -1.8572750302023784


In [6]:
# setup VQE 
# setup optimizer, use L_BFGS_B optimizer for example
lbfgs = L_BFGS_B(maxfun=1000, factr=10, iprint=10)

# setup variational form generator (generate trial circuits for VQE)
var_form = RYRZ(qubitOp.num_qubits, 5, entangler_map = {0: [1], 1:[2], 2:[3]})

# setup VQE with operator, variational form, and optimizer
vqe_algorithm = VQE(qubitOp, var_form, lbfgs, 'matrix')

backend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend, pass_manager=PassManager())

results = vqe_algorithm.run(quantum_instance)
print("Minimum value: {}".format(results['eigvals'][0].real))
print("Parameters: {}".format(results['opt_params']))

Minimum value: -1.8543764647149892
Parameters: [-0.11491214  0.32930264 -0.88261763  1.06617404  0.40859901 -1.97440033
 -1.92168204 -0.84401099  0.86926132  0.19962884  1.88259456  0.58563547
  0.2392261  -1.22095488 -1.8106901  -2.18264222  0.33053157  1.44130586
  1.16319625  3.08314583  2.86759507  1.39583915  0.89226104 -1.33970127
 -0.20377285 -1.78328421 -1.13356666  2.15515282  1.72935768 -1.02814735
  0.9198738  -0.51798203  2.25275439  2.70574515 -0.7428116   2.69334082
 -1.96540296 -2.06722665  0.19826459  1.29081156  0.03509571  0.55931109
  1.61732516 -0.3508986   1.62533384 -0.64622024  3.14159265 -1.78378164]
